# The Price is Right

## Week 8 Order of Play

Day 1: Modal.com and SpecialistAgent  
Day 2: RAG, FrontierAgent, Ensemble Agent  
Day 3: ScannerAgent, MessengerAgent  
Day 4: AutonomousPlannerAgent and DealAgentFramework  
Day 5: The Price Is Right Finale


Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)
openai = OpenAI()
MODEL = 'gpt-5-mini'

In [2]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [01:23<00:00, 27.88s/it]


In [3]:
len(deals)

30

In [4]:
deals[10].describe()

'Title: Samsung Galaxy Tab S11 Ultra 256GB 14.6" Tablet: $220 off, or up to $700 off w/ trade + free shippin\nDetails: With a qualifying trade-in, you can get up to $800 off Samsung\'s Galaxy Tab S11 Ultra tablet, dropping the price as low as $500. Without a trade-in, you\'ll get $220 off all three storage options, for the best discount we\'ve seen with no trade required. Shop Now at Samsung\nFeatures: \nURL: https://www.dealnews.com/Samsung-Galaxy-Tab-S11-Ultra-256-GB-14-6-Tablet-220-off-or-up-to-700-off-w-trade-free-shipping/21804175.html?iref=rss-c39'

### We are going to ask GPT-5-mini to summarize deals and identify their price

In [5]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [6]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [7]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Sonos Speaker Deals at Best Buy: Up to 35% off + free shipping
Details: Save up to $360 on a selection of Sonos speakers, including brand new offers as well as Geek Squad refurbs. We've pictured the brand new Sonos Architectural 6-1/2" Passive 2-Way Outdoor Speakers (Pair) for $650 ($350 off and a price low). Shop Now at Best Buy
Features: 
URL: https://www.dealnews.com/Sonos-S

In [ ]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

In [ ]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


In [8]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
from agents.scanner_agent import ScannerAgent

In [ ]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result

### Using smtplib to send an email

In [9]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()

INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has been initialized


In [10]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com")

INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
14:38:22 - LiteLLM:INFO: utils.py:3807 - 
LiteLLM completion() model= gemini-2.5-flash-lite; provider = gemini
INFO:LiteLLM:
LiteLLM completion() model= gemini-2.5-flash-lite; provider = gemini
14:38:23 - LiteLLM:INFO: utils.py:1573 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging Agent] ✅ Email sent successfully!
INFO:root:[Messaging Agent] Messaging Agent has completed
